In [4]:
from pyspark.sql import SparkSession
import getpass
username=getpass.getuser()
spark= SparkSession. \
builder. \
enableHiveSupport(). \
master('yarn'). \
appName("spark_scenarios"). \
config('spark.ui.port','0'). \
config('spark.sql.warehouse.dir',f"/user/{username}/warehouse"). \
getOrCreate()

In [3]:
spark

#### Get the total number of sms exchanged between two people.

In [7]:
sms_head=['sms_date','sender','receiver','sms_no']
sms_data=[('2020-04-01','Avinash','Vibhor','10'),
            ('2020-04-01','Vibhor','Avinash','20'),
            ('2020-04-01','Avinash','Pawan','30'),
            ('2020-04-01','Pawan','Avinash','20'),
            ('2020-04-01','Vibhor','Pawan','5'),
            ('2020-04-01','Pawan','Vibhor','8'),
            ('2020-04-01','Vibhor','Deepak','50')]


In [8]:
sms_df=spark.createDataFrame(sms_data,schema=sms_head)
sms_df.show()

+----------+-------+--------+------+
|  sms_date| sender|receiver|sms_no|
+----------+-------+--------+------+
|2020-04-01|Avinash|  Vibhor|    10|
|2020-04-01| Vibhor| Avinash|    20|
|2020-04-01|Avinash|   Pawan|    30|
|2020-04-01|  Pawan| Avinash|    20|
|2020-04-01| Vibhor|   Pawan|     5|
|2020-04-01|  Pawan|  Vibhor|     8|
|2020-04-01| Vibhor|  Deepak|    50|
+----------+-------+--------+------+



In [10]:
from pyspark.sql.functions import *
sms_df_1=sms_df.withColumn("person1",when((col("sender") < col("receiver")),col("sender")).otherwise(col("receiver"))) \
.withColumn("person2",when((col("sender") < col("receiver")),col("receiver")).otherwise(col("sender")))
sms_df_1.show()

+----------+-------+--------+------+-------+-------+
|  sms_date| sender|receiver|sms_no|person1|person2|
+----------+-------+--------+------+-------+-------+
|2020-04-01|Avinash|  Vibhor|    10|Avinash| Vibhor|
|2020-04-01| Vibhor| Avinash|    20|Avinash| Vibhor|
|2020-04-01|Avinash|   Pawan|    30|Avinash|  Pawan|
|2020-04-01|  Pawan| Avinash|    20|Avinash|  Pawan|
|2020-04-01| Vibhor|   Pawan|     5|  Pawan| Vibhor|
|2020-04-01|  Pawan|  Vibhor|     8|  Pawan| Vibhor|
|2020-04-01| Vibhor|  Deepak|    50| Deepak| Vibhor|
+----------+-------+--------+------+-------+-------+



In [11]:
sms_df_2=sms_df_1.groupBy("person1","person2","sms_date").agg(sum("sms_no").alias("Total_sms")) \
.orderBy("sms_date","person1","person2")
sms_df_2.show()

+-------+-------+----------+---------+
|person1|person2|  sms_date|Total_sms|
+-------+-------+----------+---------+
|Avinash|  Pawan|2020-04-01|     50.0|
|Avinash| Vibhor|2020-04-01|     30.0|
| Deepak| Vibhor|2020-04-01|     50.0|
|  Pawan| Vibhor|2020-04-01|     13.0|
+-------+-------+----------+---------+

